In [1]:
from __future__ import  absolute_import
# though cupy is not used but without this line, it raise errors...
import cupy as cp
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import ipdb
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils.config import opt
from data.dataset import Dataset, TestDataset, inverse_normalize
from model.faster_rcnn_vgg16 import FasterRCNNVGG16
from torch.utils import data as data_
import torch
from trainer import FasterRCNNTrainer
from utils import array_tool as at
from utils.vis_tool import visdom_bbox
from utils.eval_tool import eval_detection_voc

# fix for ulimit
# https://github.com/pytorch/pytorch/issues/973#issuecomment-346405667
import resource

rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (2048, rlimit[1]))

/datasets/home/85/285/ktl014/Fast-Pedestrian-Tracking/utils/vis_tool.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.6/site

In [3]:
# User arguments
print(opt._parse_all())

# Load dataset
dataset = TestDataset(opt, split='val')
dataloader = data_.DataLoader(dataset,
                                   batch_size=1,
                                   num_workers=opt.test_num_workers,
                                   shuffle=False,
                                   pin_memory=True
                                   )

# Construct model
faster_rcnn = FasterRCNNVGG16(mask=opt.mask)
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
trainer.load(opt.load_path)

# Predict
imgs, sizes, gt_bboxes_, gt_labels_ = next(iter(dataloader))
sizes = [sizes[0][0].item(), sizes[1][0].item()]
pred_bboxes_, pred_labels_, pred_scores_ = faster_rcnn.predict(imgs,[sizes])

# Show score
pred_img = visdom_bbox(ori_img_,
                    at.tonumpy(_bboxes[0]),
                    at.tonumpy(_labels[0]).reshape(-1),
                    at.tonumpy(_scores[0]))
plt.show()

======user config========
{'benchmark_path': None,
 'caffe_pretrain': False,
 'caffe_pretrain_path': 'checkpoints/vgg16_caffe.pth',
 'data': 'voc',
 'debug_file': '/tmp/debugf',
 'env': 'faster_rcnn',
 'epoch': 14,
 'load_path': None,
 'lr': 0.001,
 'lr_decay': 0.1,
 'mask': True,
 'mask_conv': False,
 'mask_lin': True,
 'max_size': 1000,
 'min_size': 600,
 'model_name': 'test_run',
 'num_workers': 4,
 'plot_every': 40,
 'port': 8097,
 'pretrained_model': 'vgg16',
 'roi_sigma': 1.0,
 'rpn_sigma': 3.0,
 'sparse_dense': False,
 'test_num': 10000,
 'test_num_workers': 4,
 'use_adam': True,
 'use_chainer': False,
 'use_drop': False,
 'use_simple': False,
 'voc_data_dir': 'dataset',
 'weight_decay': 0.0005}
==========end============
None
Using Masked Linear


CUDARuntimeError: cudaErrorUnknown: unknown error